In [1]:
import torch
import sys
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

repo_path = "/mnt/e/untitled folder/codebase/LoRO/LoRO"  
if os.path.exists(repo_path) and repo_path not in sys.path:
    sys.path.append(repo_path)

try:
    from utils import model_obfuscation
except ImportError as e:
    print('wrong repo_path')
    sys.exit(1)

# ==========================================
# 1. 加载目标模型 (Private Model)
# ==========================================
model_id = "Creekside/Qwen-3B-gsm8k-GRPO"
device = "cpu"
save_path = "/mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_obfuscated.pt"

print(f"正在加载模型: {model_id} ...")
# Qwen 需要 trust_remote_code=True
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# 使用 AutoModelForCausalLM 因为这是一个生成任务 (GSM8K)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).to(device)

print("模型加载完成。准备进行 LoRO 混淆...")

# ==========================================
# 2. 执行混淆 (调用仓库代码)
# ==========================================
noise_magnitude = 1

print(f"开始混淆 (Noise Magnitude: {noise_magnitude})...")
obfuscated_model = model_obfuscation(model, device=device, noise_mag=noise_magnitude, r=30)

# ==========================================
# 4. 保存混淆后的 Checkpoint
# ==========================================
print(f"正在保存混淆后的模型至: {save_path} ...")
torch.save(obfuscated_model.state_dict(), save_path)

print(f"Checkpoint Path: {os.path.abspath(save_path)}")


正在加载模型: Creekside/Qwen-3B-gsm8k-GRPO ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型加载完成。准备进行 LoRO 混淆...
开始混淆 (Noise Magnitude: 1)...
Obfuscating: model.layers.0.self_attn.q_proj
Obfuscating: model.layers.0.self_attn.k_proj
Obfuscating: model.layers.0.self_attn.v_proj
Obfuscating: model.layers.0.self_attn.o_proj
Obfuscating: model.layers.0.mlp.gate_proj
Obfuscating: model.layers.0.mlp.up_proj
Obfuscating: model.layers.0.mlp.down_proj
Obfuscating: model.layers.1.self_attn.q_proj
Obfuscating: model.layers.1.self_attn.k_proj
Obfuscating: model.layers.1.self_attn.v_proj
Obfuscating: model.layers.1.self_attn.o_proj
Obfuscating: model.layers.1.mlp.gate_proj
Obfuscating: model.layers.1.mlp.up_proj
Obfuscating: model.layers.1.mlp.down_proj
Obfuscating: model.layers.2.self_attn.q_proj
Obfuscating: model.layers.2.self_attn.k_proj
Obfuscating: model.layers.2.self_attn.v_proj
Obfuscating: model.layers.2.self_attn.o_proj
Obfuscating: model.layers.2.mlp.gate_proj
Obfuscating: model.layers.2.mlp.up_proj
Obfuscating: model.layers.2.mlp.down_proj
Obfuscating: model.layers.3.self_att

In [1]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd

# ==========================================
# 配置
# ==========================================
model_id_ft = "Creekside/Qwen-3B-gsm8k-GRPO"   # Target
model_id_base = "Qwen/Qwen2.5-3B-Instruct"     # Prior (Base)
device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Fine-Tuned Model: {model_id_ft}...")
model_ft = AutoModelForCausalLM.from_pretrained(model_id_ft, trust_remote_code=True).to(device)

print(f"Loading Base Model: {model_id_base}...")
model_base = AutoModelForCausalLM.from_pretrained(model_id_base, trust_remote_code=True).to(device)

print("\nStarting Comparison (FT vs. Base)...")
print("-" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Delta Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_base = dict(model_base.named_modules())

for name, module_ft in model_ft.named_modules():
    if isinstance(module_ft, torch.nn.Linear):
        if name in modules_base:
            module_base = modules_base[name]
            w_ft = module_ft.weight.detach()
            w_base = module_base.weight.detach()
            
            if w_ft.shape != w_base.shape:
                continue
                
            # 1. Cosine Similarity
            cos_sim = torch.nn.functional.cosine_similarity(
                w_ft.flatten(), w_base.flatten(), dim=0
            ).item()
            
            # 2. Delta (FT - Base)
            delta = w_ft - w_base
            norm_delta = torch.norm(delta).item()
            norm_base = torch.norm(w_base).item()
            
            rel_diff = norm_delta / norm_base if norm_base > 0 else 0.0
            
            print(f"{name:<50} | {cos_sim:.6f}   | {norm_delta:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, "Cos_Sim": cos_sim, 
                "Delta_Norm": norm_delta, "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Avg Cos Sim: {df['Cos_Sim'].mean():.4f} | Avg Delta Norm: {df['Delta_Norm'].mean():.4f}")

Loading Fine-Tuned Model: Creekside/Qwen-3B-gsm8k-GRPO...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Base Model: Qwen/Qwen2.5-3B-Instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Starting Comparison (FT vs. Base)...
------------------------------------------------------------------------------------------
Layer Name                                         | Cos Sim    | Delta Norm   | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj                    | 0.995474   | 7.0397       | 9.4047%
model.layers.0.self_attn.k_proj                    | 0.995679   | 3.4136       | 9.2446%
model.layers.0.self_attn.v_proj                    | 0.995532   | 1.1226       | 9.4010%
model.layers.0.self_attn.o_proj                    | 0.995545   | 4.0921       | 9.2753%
model.layers.0.mlp.gate_proj                       | 0.997714   | 11.8519       | 9.3296%
model.layers.0.mlp.up_proj                         | 0.997950   | 10.4254       | 9.3316%
model.layers.0.mlp.down_proj                       | 0.997777   | 11.1631       | 9.3132%
model.layers.1.self_attn.q_proj                    | 0.99550

In [1]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import copy
from tqdm import tqdm

# ==========================================
# 1. 配置
# ==========================================
# 你的 Base 模型 (攻击者先验)
base_model_id = "Qwen/Qwen2.5-3B-Instruct"
# 你的 GT 模型 (仅用于验证攻击结果)
target_model_id = "Creekside/Qwen-3B-gsm8k-GRPO"
# 混淆后的 Checkpoint 路径
obfuscated_checkpoint = "/mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_obfuscated.pt"
# 恢复模型的保存路径
save_path_recovered = "/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_model"

device = "cuda" if torch.cuda.is_available() else "cpu"

# === 攻击关键参数 ===
REMOVE_RANK = 30 

# ==========================================
# 2. 模型加载
# ==========================================
print(f"1. Loading Base Model (Prior): {base_model_id}...")
# 必须加 trust_remote_code=True 以支持 Qwen
recovered_model = AutoModelForCausalLM.from_pretrained(base_model_id, trust_remote_code=True).to(device)

print(f"2. Loading Obfuscated Checkpoint: {obfuscated_checkpoint}...")
if not os.path.exists(obfuscated_checkpoint):
    raise FileNotFoundError("混淆 Checkpoint 未找到，请先运行 Cell 1 生成。")
obfus_state_dict = torch.load(obfuscated_checkpoint, map_location=device)

print(f"3. Loading Ground Truth (Validation): {target_model_id}...")
gt_model = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)

# ==========================================
# 3. 执行 SVD 去噪攻击 (SVD Denoising)
# ==========================================
print(f"\nSTARTING RECOVERY (Removing Top-{REMOVE_RANK} Singular Values as Noise)...")
print("="*60)

similarities = []
relative_errors = []

# 遍历 Base 模型的每一层
# 我们需要修改 recovered_model 的权重
all_modules = list(recovered_model.named_modules())
linear_layers = [(n, m) for n, m in all_modules if isinstance(m, nn.Linear)]

progress_bar = tqdm(linear_layers, desc="Recovering")

for name, module in progress_bar:
    # 1. 构造 LoRO 保存的 key
    # 根据 LoroLinear 代码，权重保存在 "obfus_linear.weight"
    obfus_key = f"{name}.obfus_linear.weight"
    
    # 检查该层是否被混淆 (如果没在 checkpoint 里找到，说明该层未混淆，保持 Base 原样)
    if obfus_key not in obfus_state_dict:
        continue
        
    # 2. 获取数据
    W_base = module.weight.detach() # [out, in]
    W_obfus = obfus_state_dict[obfus_key].detach() # [out, in]
    
    # 3. 计算 Diff (包含 Fine-tuning 更新 + LoRO 噪声)
    # Diff = (W_base + Delta_FT + Noise) - W_base = Delta_FT + Noise
    Diff = W_obfus - W_base
    
    # 4. SVD 分解
    # 使用 float32 保证精度，Diff 形状通常是 [out_features, in_features]
    U, S, Vh = torch.linalg.svd(Diff.float(), full_matrices=False)

    print(f"Layer: {name} | {S[REMOVE_RANK - 2]}, {S[REMOVE_RANK - 1]}, {S[REMOVE_RANK]}, {S[REMOVE_RANK + 1]}")
    
    # --- 攻击核心逻辑 ---
    # 根据 LoroLinear 定义，噪声是 (B@A).T，其秩为 r (REMOVE_RANK)。
    # 且因为 noise_mag 很大，这 r 个奇异值会非常大，位于 S 的最前端。
    # 我们将前 r 个奇异值置零，即移除了 LoRO 噪声。
    S_clean = S.clone()
    S_clean[:REMOVE_RANK] = 0.0 
    
    # 5. 重构纯净的 Delta (即估算的 Delta_FT)
    Delta_Recovered = (U @ torch.diag(S_clean) @ Vh).to(W_base.dtype)
    
    # 6. 恢复权重并写入模型
    # W_rec = W_base + Delta_Recovered
    module.weight.data = W_base + Delta_Recovered
    
    # 7. 恢复 Bias
    # LoroLinear 代码中: self.obfus_linear.bias = original_linear.bias
    # Bias 没有加噪声，直接从 checkpoint 读取覆盖即可
    obfus_bias_key = f"{name}.obfus_linear.bias"
    if obfus_bias_key in obfus_state_dict and module.bias is not None:
        module.bias.data = obfus_state_dict[obfus_bias_key].detach()
    
    # ==========================
    # 验证环节 (对比 GT)
    # ==========================
    if name in dict(gt_model.named_modules()):
        W_gt = dict(gt_model.named_modules())[name].weight.detach()
        Delta_True = W_gt - W_base
        
        # 计算相似度: 我们的恢复结果 vs 真实的微调增量
        # 如果 sim 接近 1.0，说明我们成功剥离了噪声，保留了微调增量
        sim = torch.nn.functional.cosine_similarity(Delta_True.flatten(), Delta_Recovered.flatten(), dim=0).item()
        
        print(f"Layer: {name} | Similarity with GT Delta: {sim:.4f}")
        
        similarities.append(sim)
        # progress_bar.set_postfix({"Avg Sim": f"{sum(similarities)/len(similarities):.4f}", "Last Sim": f"{sim:.4f}"})

# ==========================================
# 4. 结果保存
# ==========================================

print(f"Saving recovered model to {save_path_recovered}...")
recovered_model.save_pretrained(save_path_recovered)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.save_pretrained(save_path_recovered)
print("Saved.")

1. Loading Base Model (Prior): Qwen/Qwen2.5-3B-Instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2. Loading Obfuscated Checkpoint: /mnt/e/untitled folder/codebase/LoRO_attack/loro_qwen_obfuscated.pt...
3. Loading Ground Truth (Validation): Creekside/Qwen-3B-gsm8k-GRPO...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


STARTING RECOVERY (Removing Top-30 Singular Values as Noise)...


...

Recovering: 100%|████████████████████████████████████████████████████████████████████▋| 252/253 [06:50<00:02,  2.07s/it]

Layer: model.layers.35.mlp.down_proj | Similarity with GT Delta: 0.9912
Layer: lm_head | 1332.6458740234375, 1292.28662109375, 0.055072203278541565, 0.051004305481910706


Recovering: 100%|█████████████████████████████████████████████████████████████████████| 253/253 [07:08<00:00,  1.69s/it]

Layer: lm_head | Similarity with GT Delta: 0.0000
Saving recovered model to /mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_model...


Saved.


In [1]:
import torch
from transformers import AutoModelForCausalLM
import pandas as pd
import os

# ==========================================
# 配置：对比 Ground Truth (Target) 与 恢复模型 (Recovered)
# ==========================================

target_model_id = "Creekside/Qwen-3B-gsm8k-GRPO" 
recovered_model_path = "/mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_model"

device = "cpu" # 对比只需 CPU 即可，省显存

print(f"Loading Target Model (GT): {target_model_id}...")
try:
    model_gt = AutoModelForCausalLM.from_pretrained(target_model_id, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading GT model: {e}")
    # 如果显存不够，可以尝试加 device_map="cpu" 或 torch_dtype=torch.float16

print(f"Loading Recovered Model: {recovered_model_path}...")

try:
    model_recovered = AutoModelForCausalLM.from_pretrained(recovered_model_path, trust_remote_code=True).to(device)
except Exception as e:
    print(f"Error loading Recovered model: {e}")
    raise e

print("\nStarting Comparison (Target GT vs. Recovered)...")
print("*" * 90)
print(f"{'Layer Name':<50} | {'Cos Sim':<10} | {'Diff Norm':<12} | {'Rel Diff (%)':<12}")
print("-" * 90)

results = []
modules_recovered = dict(model_recovered.named_modules())

# 遍历 GT 模型的层
for name, module_gt in model_gt.named_modules():
    if isinstance(module_gt, torch.nn.Linear):
        if name in modules_recovered:
            module_rec = modules_recovered[name]
            
            # 获取权重
            w_gt = module_gt.weight.detach()
            w_rec = module_rec.weight.detach()
            
            # 检查形状
            if w_gt.shape != w_rec.shape:
                continue
                
            # 1. 计算余弦相似度
            cos_sim = torch.nn.functional.cosine_similarity(
                w_gt.flatten(), w_rec.flatten(), dim=0
            ).item()
            
            # 2. 计算差异 (Diff = GT - Recovered)
            # 如果攻击完美，Diff 应该全是 0
            diff = w_gt - w_rec
            norm_diff = torch.norm(diff).item()
            norm_gt = torch.norm(w_gt).item()
            
            # 相对差异
            rel_diff = norm_diff / norm_gt if norm_gt > 0 else 0.0
            
            # 打印结果 (只打印部分层，或者差异较大的层)
            # 如果 Cos Sim < 0.99 或者 Diff 比较大，说明恢复有问题
            print(f"{name} | {cos_sim:.6f}   | {norm_diff:.4f}       | {rel_diff*100:.4f}%")
            
            results.append({
                "Layer": name, 
                "Cos_Sim": cos_sim, 
                "Diff_Norm": norm_diff, 
                "Rel_Diff": rel_diff
            })

df = pd.DataFrame(results)
print("-" * 90)
print(f"Summary (Target vs Recovered):")
print(f"Avg Cosine Similarity: {df['Cos_Sim'].mean():.6f} (Target: 1.0)")
print(f"Avg Relative Diff:     {df['Rel_Diff'].mean()*100:.6f}% (Target: 0.0%)")
print("-" * 90)


Loading Target Model (GT): Creekside/Qwen-3B-gsm8k-GRPO...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading Recovered Model: /mnt/e/untitled folder/codebase/LoRO_attack/recovered_qwen_model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]


Starting Comparison (Target GT vs. Recovered)...
******************************************************************************************
Layer Name                                         | Cos Sim    | Diff Norm    | Rel Diff (%)
------------------------------------------------------------------------------------------
model.layers.0.self_attn.q_proj | 0.999762   | 1.2662       | 1.6910%
model.layers.0.self_attn.k_proj | 0.999424   | 1.2366       | 3.3498%
model.layers.0.self_attn.v_proj | 0.999404   | 0.4067       | 3.4042%
model.layers.0.self_attn.o_proj | 0.999720   | 0.7519       | 1.7044%
model.layers.0.mlp.gate_proj | 1.002602   | 1.6153       | 1.2722%
model.layers.0.mlp.up_proj | 1.002538   | 1.4156       | 1.2673%
model.layers.0.mlp.down_proj | 1.002535   | 1.5171       | 1.2661%
model.layers.1.self_attn.q_proj | 0.999702   | 1.0116       | 1.7124%
model.layers.1.self_attn.k_proj | 0.999431   | 1.0295       | 3.3604%
model.layers.1.self_attn.v_proj | 0.999406   | 0.4548  

In [ ]:
# FT with Knockoff 
# then test accuracy